# SQL Interview Questions - Lab

## Introduction

In this lab, we'll test our SQL skills against some real-world interview questions from major companies!

## Objectives

You will be able to:

* Write SQL queries to filter and order results
* Decide and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Getting Started

In this lab, we'll see four different interview questions that test your SQL knowledge. We didn't write these questions -- instead, we found them out in the real-world. These are questions that have been used in the past by major technology companies such as Facebook, Amazon, and Twitter. Our goal here isn't to memorize the questions or anything like that -- after all, it's extremely unlikely that these questions are still in use, now that they've become publicly available on the interwebs. Instead, our goal is to treat these questions as if they are the real thing, and give us some insight into the types of questions we'll need to be able to answer in order pass an interview involving SQL. 

If these questions seem hard to you, don't sweat it, they're supposed to be tough! These are meant to help you identify any areas of knowledge where you still need to grow! Use these questions as a way to see where your SQL knowledge is strong, and where it's a bit weak. Then, go study and **practice** in the areas where you still need work!

### A Note on Answering These Questions

Since these are interview questions, they'll almost always be posed as hypotheticals. This means that you won't have a real database to work with and test your code on. This also means that there are multiple different solutions to any given problem listed here. Be sure to doublecheck the code you write for bugs and errors. It's much harder to write bug-free code when you aren't able to test it against a database!

If these questions seem hard, that's normal. These are real questions that have been reported to online forums from job seekers at major companies. Obviously, it's unlikely that they're still in use at these companies, but they still represent a great way for us to test our skills against the kinds of questions we can expect to be asked in an interview!

## Question 1

From Facebook:

Assume we have a table of employee information, which includes salary information. Write a query to find the names and salaries of the top 5 highest paid employees, in descending order.

In [6]:
"""SELECT e.full_name, e.salary
   FROM employees e
   ORDER BY salary DESC
   LIMIT 5"""

'SELECT e.full_name, e.salary\n   FROM employees e\n   ORDER BY salary DESC\n   LIMIT 5'

## Question 2

From Amazon:

Assume we have two SQL tables: `authors` and `books`. The authors table has a few million rows, and looks like this: 

| author_name | book_name |
|:-----------:|:---------:|
|   author_1  |   book_1  |
|   author_1  |   book_2  |
|   author_2  |   book_3  |
|   author_2  |   book_4  |
|   author_2  |   book_5  |
|   author_3  |   book_6  |

The books dataset also has a few million rows, and looks like this:

| book_name | copies_sold |
|:---------:|:-----------:|
|   book_1  |    10000    |
|   book_2  |     2575    |
|   book_3  |    60000    |
|   book_4  |    98000    |
|   book_5  |     5250    |
|   book_6  |    19775    |

Write an SQL query that shows the top 3 authors who sold the most total books. 

In [7]:
# Your code here
"""SELECT a.author_name
   FROM authors a
   LEFT JOIN books b USING (book_name)
   GROUP BY a.author_name
   ORDER BY SUM(b.copies_sold) DESC
   LIMIT 3
"""

'SELECT a.author_name\n   FROM authors a\n   LEFT JOIN books b USING (book_name)\n   GROUP BY a.author_name\n   ORDER BY SUM(b.copies_sold) DESC\n   LIMIT 3\n'

## Question 3

From Amazon:

Assume you have two tables, `customers` and `orders`. Write a SQL query to select all customers who purchased at least 2 items on two separate days. 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('question3.db')
cur = conn.cursor()

In [2]:
# Create sample tables
cur.execute("""CREATE TABLE IF NOT EXISTS customers (
                customer_id INTEGER PRIMARY KEY
              , customer_name TEXT)""")

cur.execute("""CREATE TABLE IF NOT EXISTS orders (
                order_id INTEGER PRIMARY KEY
              , customer_id INTEGER
              , order_date TEXT
              , item_count INTEGER)""")

In [ ]:
# Insert sample values into Customer table
#values = ['Jessica Miles', 'Fred Rogers', 'Andrew Cuomo', 'Donald Trump']

#[cur.execute(f"""INSERT INTO customers (customer_name) 
#        VALUES ('{val}')""") for val in values]

In [5]:
sql_str = """SELECT * FROM customers"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customer_id,customer_name
0,1,Jessica Miles
1,2,Fred Rogers
2,3,Andrew Cuomo
3,4,Donald Trump


In [21]:
# Insert values into Orders table
#values = [[1, '2020-03-04', 3]
#         ,[1, '2020-03-10', 5]
#         ,[2, '2020-03-01', 1]
#         ,[2, '2020-03-31', 2]
#         ,[4, '2020-04-01', 1]
#         ,[4, '2020-04-01', 1]
#         ,[4, '2020-04-01', 1]
#         ,[4, '2020-04-10', 1]
#         ,[4, '2020-04-10', 1]
#         ,[3, '2020-05-01', 15]
#         ]

#[cur.execute(f"""INSERT INTO orders (customer_id, order_date, item_count) 
#        VALUES ({str(val)[1:-1]})""") for val in values]

In [6]:
sql_str = """SELECT * FROM orders"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,order_id,customer_id,order_date,item_count
0,1,1,2020-03-04,3
1,2,1,2020-03-10,5
2,3,2,2020-03-01,1
3,4,2,2020-03-31,2
4,5,4,2020-04-01,1
5,6,4,2020-04-01,1
6,7,4,2020-04-01,1
7,8,4,2020-04-10,1
8,9,4,2020-04-10,1
9,10,3,2020-05-01,15


In [7]:
conn.commit()

In [51]:
# My proposed solution. Accounts for the fact that customers may have placed 
#    multiple orders on the same day

# Jessica ordered at least 2 items on two different days, so should return
# Mr. Rogers ordered 2 items on  only one day, so should NOT return
# Cuomo ordered lots of items on a single day and should NOT return
# Donald Trump ordered only one item at a time, but multiple orders per day
#     which add up to 2 or more, and should return

sql_str = """SELECT DISTINCT(c.customer_name)
               FROM customers c
               INNER JOIN (SELECT customer_id, order_date, SUM(item_count)
                           FROM orders
                           GROUP BY customer_id, order_date
                           HAVING SUM(item_count) >= 2) items
                        ON items.customer_id = c.customer_id
               GROUP BY c.customer_id, c.customer_name
               HAVING COUNT(items.order_date) = 2
               ;"""


df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df


,customer_name
0,Jessica Miles
1,Donald Trump


In [7]:
# Original solution from Flatiron. I adjusted column names to match my test db
#
# - In the main query, we can't use count() because there is no group by clause
#   Need to add a GROUP BY clause and use HAVING instead of WHERE
# - Order date is missing from the subquery, so using it in the main query won't work
# - In the subquery, you wouldn't join tables by order number/id, 
#      since a customer would have multiple orders. In a well-designed db,
#      order_id wouldn't exist in the customers table.
# - The subquery isn't grouping by date, it's assuming each customer only 
#      made one order on each day

# The solution as written doesn't really work. See notes above.
sql_str_orig = """SELECT customer_name
                       , COUNT(DISTINCT o.order_date) as NumOrderDates 
                  FROM (SELECT c.customer_name
                             , o.item_count 
                        FROM customers c 
                        JOIN orders o ON c.order_id = o.order_id 
                        WHERE o.item_count > 1) 
                  WHERE NumOrderDates > 1;"""

# I adjusted the original solution query below so it would WORK, 
# but it still doesn't account for the fact that a customer may place 
# multiple orders on the same day.
sql_str = """SELECT customer_name
                  , COUNT(DISTINCT order_date) as NumOrderDates
             FROM (SELECT customer_name
                        , o.item_count
                        , o.order_date
                   FROM customers c 
                   JOIN orders o ON c.customer_id = o.customer_id
                   WHERE o.item_count > 1)
             GROUP BY customer_name
             HAVING NumOrderDates > 1;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customer_name,NumOrderDates
0,Jessica Miles,2


## Question 4

From Twitter:

A company uses 2 data tables, `Employee` and `Department`, to store data about its employees and departments. 

Table Name: Employee   
Attributes:   
ID Integer,   
NAME String,   
SALARY Integer,   
DEPT_ID Integer   

Table Name: Department   
Attributes:   
DEPT_ID Integer,   
NAME String,   
LOCATION String   

Write a query to print the respective Department Name and number of employees for all departments in the Department table (even unstaffed ones). 

Sort your result in descending order of employees per department; if two or more departments have the same number of employees, then sort those departments alphabetically by Department Name.

In [ ]:
# Your code here
"""SELECT d.NAME AS Department_Name
        , COUNT(e.ID) AS Employee_Count
   FROM Department d
   LEFT JOIN Employee e USING (DEPT_ID)
   GROUP BY d.DEPT_ID, d.NAME
   ORDER BY Employee_Count DESC, Department_Name
"""

## Summary

In this lab, we tested our knowledge of SQL queries against some real-world interview questions!